# EDA


- The os module has a perfect method to list files in a directory.
- Pandas json normalize could work here but is not necessary to convert the JSON data to a dataframe.
- You may need a nested for-loop to access each sale!
- We've put a lot of time into creating the structure of this repository, and it's a good example for future projects.  In the file functions_variables.py, there is an example function that you can import and use.  If you have any variables, functions or classes that you want to make, they can be put in the functions_variables.py file and imported into a notebook.  Note that only .py files can be imported into a notebook. If you want to import everything from a .py file, you can use the following:
```python
from functions_variables import *
```
If you just import functions_variables, then each object from the file will need to be prepended with "functions_variables"\
Using this .py file will keep your notebooks very organized and make it easier to reuse code between notebooks.

In [32]:
# (this is not an exhaustive list of libraries)
import pandas as pd
import numpy as np
import os
import json
from pprint import pprint
from functions_variables import encode_tags

## Data Importing

In [33]:
# load one file first to see what type of data you're dealing with and what attributes it has

import json

# Path to your JSON file
file_path = "/Users/snoopy/GitHub/LHL-Project-3-Midterm-Project/data/AK_Juneau_0.json"

# Open and read the JSON file
with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# Print the entire JSON with indentation for readability
print(json.dumps(data, indent=4))

{
    "status": 200,
    "data": {
        "total": 8,
        "count": 8,
        "results": [
            {
                "primary_photo": {
                    "href": "https://ap.rdcpix.com/07097d34c98a59ebb799688986ef271dl-m2867511255s-w1024_h768.jpg"
                },
                "last_update_date": "2023-09-19T20:52:50Z",
                "source": {
                    "plan_id": null,
                    "agents": [
                        {
                            "office_name": "EXP Realty LLC - Southeast Alaska"
                        },
                        {
                            "office_name": "Non-Member Office"
                        }
                    ],
                    "spec_id": null,
                    "type": "mls"
                },
                "tags": [
                    "carport",
                    "community_outdoor_space",
                    "cul_de_sac",
                    "family_room",
                    "hardwood_fl

In [34]:
# loop over all files and put them into a dataframe

import os
import json
import pandas as pd

# Path to folder containing JSON files
folder_path = "/Users/snoopy/GitHub/LHL-Project-3-Midterm-Project/data"

# List to store all property data
all_properties = []

# Set to track unique keys (for structure verification)
schema_set = set()

# Function to safely extract values from nested dictionaries
def safe_get(d, keys, default=None):
    """Safely fetches a nested key from a dictionary."""
    for key in keys:
        if isinstance(d, dict):
            d = d.get(key, default)
        else:
            return default
    return d

# Function to flatten property data
def flatten_property(property_item):
    """Extracts relevant fields and flattens nested dictionaries."""
    if property_item is None:
        return {}

    return {
        "property_id": safe_get(property_item, ["property_id"]),
        "status": safe_get(property_item, ["status"]),
        "list_date": safe_get(property_item, ["list_date"]),
        "list_price": safe_get(property_item, ["list_price"]),
        "sold_date": safe_get(property_item, ["description", "sold_date"]),
        "sold_price": safe_get(property_item, ["description", "sold_price"]),
        "year_built": safe_get(property_item, ["description", "year_built"]),
        "beds": safe_get(property_item, ["description", "beds"]),
        "baths": safe_get(property_item, ["description", "baths"]),
        "sqft": safe_get(property_item, ["description", "sqft"]),
        "lot_sqft": safe_get(property_item, ["description", "lot_sqft"]),
        "garage": safe_get(property_item, ["description", "garage"]),
        "type": safe_get(property_item, ["description", "type"]),
        "address": safe_get(property_item, ["location", "address", "line"]),
        "city": safe_get(property_item, ["location", "address", "city"]),
        "state": safe_get(property_item, ["location", "address", "state"]),
        "postal_code": safe_get(property_item, ["location", "address", "postal_code"]),
        "latitude": safe_get(property_item, ["location", "address", "coordinate", "lat"]),
        "longitude": safe_get(property_item, ["location", "address", "coordinate", "lon"]),
        "primary_photo": safe_get(property_item, ["primary_photo", "href"]),
    }

# Loop through JSON files in folder
for file_name in os.listdir(folder_path):
    if file_name.endswith(".json"):  # Ensure it's a JSON file
        file_path = os.path.join(folder_path, file_name)
        
        with open(file_path, "r", encoding="utf-8") as f:
            try:
                data = json.load(f)
                if "data" in data and "results" in data["data"]:
                    for property_item in data["data"]["results"]:
                        flattened_data = flatten_property(property_item)
                        all_properties.append(flattened_data)
                        schema_set.add(frozenset(flattened_data.keys()))  # Store keys for consistency check
                
            except json.JSONDecodeError:
                print(f"Error reading {file_name}")

# Check for structural consistency
if len(schema_set) == 1:
    print("All files have the same structure.")
else:
    print("Files have different structures.")

# Convert to DataFrame
df = pd.DataFrame(all_properties)

# Display first few rows
print(df.head())


All files have the same structure.
  property_id status                    list_date  list_price   sold_date  \
0  3300959879   sold  2023-11-27T05:15:07.000000Z    334900.0  2024-01-16   
1  3149247078   sold  2023-11-23T11:38:45.000000Z    185000.0  2024-01-16   
2  4686076173   sold  2023-12-16T02:39:18.000000Z    197000.0  2024-01-12   
3  4085014275   sold  2023-10-24T14:15:02.000000Z    295000.0  2024-01-12   
4  3743385520   sold  2023-11-15T17:43:35.000000Z    105000.0  2024-01-12   

   sold_price  year_built  beds  baths    sqft  lot_sqft  garage  \
0    345000.0      2001.0   3.0    2.0  1539.0   22651.0     2.0   
1    196000.0      1900.0   3.0    NaN  2429.0    2614.0     3.0   
2    205000.0      1971.0   3.0    2.0  1120.0   13504.0     NaN   
3    295000.0      2013.0   3.0    3.0  2400.0    2688.0     1.0   
4    106000.0      1900.0   3.0    NaN  1478.0     871.0     NaN   

            type          address        city         state postal_code  \
0  single_family  

## Data Cleaning and Wrangling

At this point, ensure that you have all sales in a dataframe.
- Take a quick look at your data (i.e. `.info()`, `.describe()`) - what do you see?
- Is each cell one value, or do some cells have lists?
- What are the data types of each column?
- Some sales may not actually include the sale price (target).  These rows should be dropped.
- There are a lot of NA/None values.  Should these be dropped or replaced with something?
    - You can drop rows or use various methods to fills NA's - use your best judgement for each column 
    - i.e. for some columns (like Garage), NA probably just means no Garage, so 0
- Drop columns that aren't needed
    - Don't keep the list price because it will be too close to the sale price. Assume we want to predict the price of houses not yet listed

In [35]:
# Check data types and missing values
df.info()

# Summary statistics
df.describe(include="all")

# Check if any columns contain lists
df.applymap(lambda x: isinstance(x, list)).sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8159 entries, 0 to 8158
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   property_id    8159 non-null   object 
 1   status         8159 non-null   object 
 2   list_date      7752 non-null   object 
 3   list_price     7721 non-null   float64
 4   sold_date      8159 non-null   object 
 5   sold_price     6716 non-null   float64
 6   year_built     7316 non-null   float64
 7   beds           7504 non-null   float64
 8   baths          7980 non-null   float64
 9   sqft           7323 non-null   float64
 10  lot_sqft       6991 non-null   float64
 11  garage         4448 non-null   float64
 12  type           8125 non-null   object 
 13  address        8144 non-null   object 
 14  city           8154 non-null   object 
 15  state          8159 non-null   object 
 16  postal_code    8159 non-null   object 
 17  latitude       7909 non-null   float64
 18  longitud

property_id      0
status           0
list_date        0
list_price       0
sold_date        0
sold_price       0
year_built       0
beds             0
baths            0
sqft             0
lot_sqft         0
garage           0
type             0
address          0
city             0
state            0
postal_code      0
latitude         0
longitude        0
primary_photo    0
dtype: int64

In [38]:
# Drop rows where 'sold_price', 'lot_sqft', or 'year_built' is missing
df = df.dropna(subset=["sold_price", "lot_sqft", "year_built", "list_price","beds", "baths", "sqft"])

# Drop unnecessary columns
df = df.drop(columns=["primary_photo"], errors="ignore")

# replace nulls with 0
df["garage"] = df["garage"].fillna(0)

# Convert data types
df["sold_price"] = df["sold_price"].astype(float)
df["lot_sqft"] = pd.to_numeric(df["lot_sqft"], errors="coerce")
df["year_built"] = df["year_built"].astype(int)


df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 5131 entries, 0 to 8158
Data columns (total 19 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   property_id  5131 non-null   object 
 1   status       5131 non-null   object 
 2   list_date    5131 non-null   object 
 3   list_price   5131 non-null   float64
 4   sold_date    5131 non-null   object 
 5   sold_price   5131 non-null   float64
 6   year_built   5131 non-null   int64  
 7   beds         5131 non-null   float64
 8   baths        5131 non-null   float64
 9   sqft         5131 non-null   float64
 10  lot_sqft     5131 non-null   float64
 11  garage       5131 non-null   float64
 12  type         5131 non-null   object 
 13  address      5126 non-null   object 
 14  city         5126 non-null   object 
 15  state        5131 non-null   object 
 16  postal_code  5131 non-null   object 
 17  latitude     5112 non-null   float64
 18  longitude    5112 non-null   float64
dtypes: float64(

### Dealing with Tags

Consider the fact that with tags, there are a lot of categorical variables.
- How many columns would we have if we OHE tags, city and state?
- Perhaps we can get rid of tags that have a low frequency.

### Dealing with Cities

- Sales will vary drastically between cities and states.  Is there a way to keep information about which city it is without OHE?
- Could we label encode or ordinal encode?  Yes, but this may have undesirable effects, giving nominal data ordinal values.
- What we can do is use our training data to encode the mean sale price by city as a feature (a.k.a. Target Encoding)
    - We can do this as long as we ONLY use the training data - we're using the available data to give us a 'starting guess' of the price for each city, without needing to encode city explicitly
- If you replace cities or states with numerical values (like the mean price), make sure that the data is split so that we don't leak data into the training selection. This is a great time to train test split. Compute on the training data, and join these values to the test data
- Note that you *may* have cities in the test set that are not in the training set. You don't want these to be NA, so maybe you can fill them with the overall mean

In [15]:
# perform train test split here
# do something with state and city

## Extra Data - STRETCH

> This doesn't need to be part of your Minimum Viable Product (MVP). We recommend you write a functional, basic pipeline first, then circle back and join new data if you have time

> If you do this, try to write your downstream steps in a way it will still work on a dataframe with different features!

- You're not limited to just using the data provided to you. Think/ do some research about other features that might be useful to predict housing prices. 
- Can you import and join this data? Make sure you do any necessary preprocessing and make sure it is joined correctly.
- Example suggestion: could mortgage interest rates in the year of the listing affect the price? 

## EDA/ Visualization

Remember all of the EDA that you've been learning about?  Now is a perfect time for it!
- Look at distributions of numerical variables to see the shape of the data and detect outliers.    
    - Consider transforming very skewed variables
- Scatterplots of a numerical variable and the target go a long way to show correlations.
- A heatmap will help detect highly correlated features, and we don't want these.
    - You may have too many features to do this, in which case you can simply compute the most correlated feature-pairs and list them
- Is there any overlap in any of the features? (redundant information, like number of this or that room...)

In [16]:
# perform EDA here

## Scaling and Finishing Up

Now is a great time to scale the data and save it once it's preprocessed.
- You can save it in your data folder, but you may want to make a new `processed/` subfolder to keep it organized